이미지를 코랩 로컬로 리사이즈 및 복사

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import os
from PIL import Image
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm

# ==============================================================================
# 1. 설정
# ==============================================================================
# CSV 파일 경로 (검증된 파일 사용)
CSV_PATH = '/content/drive/MyDrive/REM_project/github/dataset/split_data/master_dataset_verified.csv'

# 원본 이미지가 있는 구글 드라이브 경로
DRIVE_IMG_ROOT = '/content/drive/MyDrive/mimic_cxr_data/raw'

# 복사해둘 Colab 로컬 경로 (학습할 때 여기를 바라보게 됨)
LOCAL_SAVE_DIR = '/content/train_images'

# 이미지 크기 (DenseNet 입력 크기)
IMG_SIZE = (224, 224)

# ==============================================================================
# 2. 준비
# ==============================================================================
# 저장할 폴더 생성
os.makedirs(LOCAL_SAVE_DIR, exist_ok=True)

# 데이터 로드
df = pd.read_csv(CSV_PATH)
print(f"🚀 총 {len(df)}장의 이미지를 로컬로 리사이즈 및 복사합니다...")

# ==============================================================================
# 3. 변환 함수 (병렬 처리용)
# ==============================================================================
def process_and_save_image(row_path):
    try:
        # 1. 원본 경로 (드라이브)
        full_src_path = os.path.join(DRIVE_IMG_ROOT, str(row_path))

        # 2. 저장할 파일명 (파일명만 떼어냄)
        filename = os.path.basename(str(row_path))
        # 혹시 확장자가 없거나 다르면 jpg로 통일
        filename = os.path.splitext(filename)[0] + ".jpg"

        full_dst_path = os.path.join(LOCAL_SAVE_DIR, filename)

        # 3. 이미 존재하면 건너뜀 (재실행 시 시간 절약)
        if os.path.exists(full_dst_path):
            return True

        # 4. 이미지 열기 및 리사이즈
        with Image.open(full_src_path) as img:
            img = img.convert('RGB')
            img = img.resize(IMG_SIZE, Image.LANCZOS)
            # 5. 로컬에 저장 (압축률 85 정도면 충분)
            img.save(full_dst_path, "JPEG", quality=85)

        return True
    except Exception as e:
        # 에러 나면 그냥 넘어감 (나중에 확인 가능)
        return False

# ==============================================================================
# 4. 실행 (고속 병렬 처리)
# ==============================================================================
paths = df['path'].tolist()

# max_workers=16 정도로 병렬 처리
with ThreadPoolExecutor(max_workers=16) as executor:
    results = list(tqdm(executor.map(process_and_save_image, paths), total=len(paths), desc="Moving & Resizing"))

success_count = sum(results)
print("-" * 50)
print(f"✅ 작업 완료!")
print(f"   - 성공: {success_count}장")
print(f"   - 저장 위치: {LOCAL_SAVE_DIR}")
print("-" * 50)

데이터 정의

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch

# ==============================================================================
# 1. 설정 및 데이터 로드
# ==============================================================================
BASE_DIR = '/content/drive/MyDrive/REM_project/github/dataset/split_data/'
DATA_PATH = BASE_DIR + 'master_dataset_verified.csv'
IMG_ROOT = '/content/train_images/'

print("📂 데이터 로드 및 분할 중...")
df = pd.read_csv(DATA_PATH)
subject_ids = df['subject_id'].unique()

# Train / Val / Test 분할
train_ids, temp_ids = train_test_split(subject_ids, test_size=0.2, random_state=42)
val_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=42)

train_df = df[df['subject_id'].isin(train_ids)].reset_index(drop=True)
val_df = df[df['subject_id'].isin(val_ids)].reset_index(drop=True)

print(f"   - Train: {len(train_df)}개")
print(f"   - Val: {len(val_df)}개")

# ==============================================================================
# 2. Tabular 데이터 설정
# ==============================================================================
TABULAR_COLS = [
    'gender', 'anchor_age', 'temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp',
    'WBC', 'BNP'
]

scaler_mean = train_df[TABULAR_COLS].mean()
scaler_std = train_df[TABULAR_COLS].std().replace(0, 1)

# ==============================================================================
# 3. Dataset 클래스
# ==============================================================================
class MultimodalDataset(Dataset):
    def __init__(self, dataframe, img_root, transform=None):
        self.df = dataframe
        self.img_root = img_root
        self.transform = transform
        self.target_cols = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion', 'Pneumonia']
        self.tabular_cols = TABULAR_COLS
        self.mean = scaler_mean
        self.std = scaler_std

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # 이미지 로드
        original_path = str(row['path'])
        filename = os.path.basename(original_path)
        filename = os.path.splitext(filename)[0] + ".jpg"
        full_img_path = os.path.join(self.img_root, filename)

        try:
            image = Image.open(full_img_path).convert('RGB')
        except:
            image = Image.new('RGB', (224, 224), (0, 0, 0))

        if self.transform:
            image = self.transform(image)

        # 바이탈 데이터
        tabular = row[self.tabular_cols].values.astype(np.float32)
        tabular = (tabular - self.mean.values.astype(np.float32)) / self.std.values.astype(np.float32)

        # 라벨
        labels = row[self.target_cols].values.astype(np.float32)

        return image, torch.tensor(tabular), torch.tensor(labels)

# ==============================================================================
# 4. DataLoader (★ 중요: 의료 데이터에 안전한 증강 적용)
# ==============================================================================
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

# [수정됨] 흉부 엑스레이에 안전한 변환만 적용
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),

    # 1. 회전: 환자가 삐딱하게 누워있을 수 있으므로 -10~10도 정도는 괜찮음
    transforms.RandomRotation(degrees=10),

    # 2. 이동 및 스케일: 촬영 위치가 조금 다르거나 줌이 다를 수 있음 (이동 5%, 축소/확대 10%)
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05), scale=(0.9, 1.1)),

    # 3. 텐서 변환 및 정규화
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])

# 검증 데이터는 원본 유지
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])

train_dataset = MultimodalDataset(train_df, IMG_ROOT, train_transform)
val_dataset = MultimodalDataset(val_df, IMG_ROOT, val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

print(f"✅ 안전한 증강 적용 완료! Train Loader 배치 개수: {len(train_loader)}")

모델 정의

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

# 1. Gated Cross Attention (기존 유지)
class GatedCrossAttention(nn.Module):
    def __init__(self, d_model=1024):
        super(GatedCrossAttention, self).__init__()
        self.query_proj = nn.Linear(d_model, d_model)
        self.key_proj = nn.Linear(d_model, d_model)
        self.value_proj = nn.Linear(d_model, d_model)
        self.gate = nn.Parameter(torch.tensor(0.0))
        self.out_proj = nn.Linear(d_model, d_model)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, img_feat, tab_feat):
        B, C, H, W = img_feat.shape
        img_flat = img_feat.view(B, C, -1).permute(0, 2, 1)
        vital_query = tab_feat.unsqueeze(1)

        Q = self.query_proj(vital_query)
        K = self.key_proj(img_flat)
        V = self.value_proj(img_flat)

        scores = torch.matmul(Q, K.transpose(-2, -1)) / (C ** 0.5)
        attn_map = F.softmax(scores, dim=-1)
        context = torch.matmul(attn_map, V)

        gate_value = torch.sigmoid(self.gate)
        fused_feat = vital_query + (gate_value * context)
        fused_feat = self.norm(self.out_proj(fused_feat))

        return fused_feat.squeeze(1), attn_map.view(B, H, W)

# 2. 전체 모델 조립 (Dropout 강화 + 은닉층 추가)
class DenseNetFusionModel(nn.Module):
    def __init__(self, num_tabular_features, num_classes=1):
        super(DenseNetFusionModel, self).__init__()

        # (1) 이미지 인코더
        self.img_enc = models.densenet121(weights='DEFAULT')
        self.img_enc_features = self.img_enc.features

        # (2) 바이탈 인코더 (Dropout 추가)
        self.tab_enc = nn.Sequential(
            nn.Linear(num_tabular_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),      # 과적합 방지
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5)       # 과적합 방지
        )

        # (3) Fusion
        self.fusion = GatedCrossAttention(d_model=1024)

        # (4) ★ 수정된 분류기 (은닉층 2개 추가) ★
        # 구조: 1024 -> [512] -> [256] -> 6
        self.classifier = nn.Sequential(
            # 은닉층 1
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),     # 강력한 Dropout

            # 은닉층 2
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),     # 강력한 Dropout

            # 출력층
            nn.Linear(256, num_classes)
        )

    def forward(self, x_img, x_tab):
        img_feat = self.img_enc_features(x_img)
        tab_feat = self.tab_enc(x_tab)

        fused_feat, attn_map = self.fusion(img_feat, tab_feat)
        logits = self.classifier(fused_feat)

        return logits, attn_map

학습 시작

In [ ]:
import torch.optim as optim
from sklearn.metrics import roc_auc_score
import time
from tqdm import tqdm

# 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_EPOCHS = 15
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
NUM_CLASSES = 6
TABULAR_DIM = 10

# 모델 생성
model = DenseNetFusionModel(num_classes=NUM_CLASSES, num_tabular_features=TABULAR_DIM)
model = model.to(device)

# Loss & Optimizer
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-3) # L2 규제 유지
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=2)

# 학습 함수
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    loop = tqdm(loader, leave=True)
    for images, tabular, labels in loop:
        images = images.to(device).float()
        tabular = tabular.to(device).float()
        labels = labels.to(device).float()

        optimizer.zero_grad()
        outputs, _ = model(images, tabular)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        loop.set_description(f"Train Loss: {loss.item():.4f}")
    return running_loss / len(loader.dataset)

# 검증 함수
def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_targets, all_preds = [], []
    with torch.no_grad():
        for images, tabular, labels in loader:
            images = images.to(device).float()
            tabular = tabular.to(device).float()
            labels = labels.to(device).float()

            outputs, _ = model(images, tabular)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)

            all_targets.append(labels.cpu().numpy())
            all_preds.append(torch.sigmoid(outputs).cpu().numpy())

    epoch_loss = running_loss / len(loader.dataset)
    all_targets = np.concatenate(all_targets)
    all_preds = np.concatenate(all_preds)
    try:
        auroc = roc_auc_score(all_targets, all_preds, average='macro')
    except:
        auroc = 0.5
    return epoch_loss, auroc

# 메인 실행
best_auroc = 0.0
print("🚀 Re-Training (Safe Augmentation + Dropout 0.5 + 2 Hidden Layers)...")

for epoch in range(NUM_EPOCHS):
    train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_auroc = validate(model, val_loader, criterion, device)

    scheduler.step(val_auroc)

    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val AUROC: {val_auroc:.4f}")

    if val_auroc > best_auroc:
        best_auroc = val_auroc
        torch.save(model.state_dict(), os.path.join(BASE_DIR, "best_fusion_model_v3.pth"))
        print(f"✅ Best Model Saved! (AUROC: {val_auroc:.4f})")

    print("-" * 50)

이미지 온리 모델 정의

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class ImageOnlyDenseNet(nn.Module):
    def __init__(self, num_classes=1):
        super().__init__()
        # 퓨전 모델과 동일한 가중치 사용
        self.backbone = models.densenet121(weights='DEFAULT')

        # DenseNet의 마지막 분류기 층(1024 -> num_classes)을
        # 퓨전 모델과 똑같은 "강력한 분류기"로 교체
        num_ftrs = self.backbone.classifier.in_features # 1024

        self.backbone.classifier = nn.Sequential(
            # 은닉층 1
            nn.Linear(num_ftrs, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),  # 퓨전 모델과 동일

            # 은닉층 2
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),  # 퓨전 모델과 동일

            # 출력층
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.backbone(x)

이미지 온리 학습 시작

In [ ]:
import torch.optim as optim
from sklearn.metrics import roc_auc_score
import os
from tqdm import tqdm
import numpy as np

# 설정 (퓨전 모델과 동일하게 유지)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_EPOCHS = 15
LEARNING_RATE = 1e-4
NUM_CLASSES = 6
BASE_DIR = '/content/drive/MyDrive/REM_project/github/dataset/split_data/'

# 모델 생성
model_img = ImageOnlyDenseNet(num_classes=NUM_CLASSES).to(device)

# Loss & Optimizer
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model_img.parameters(), lr=LEARNING_RATE, weight_decay=1e-3) # L2 규제 동일 적용
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=2)

# 학습 함수 (Tabular 데이터는 받지만 사용하지 않음)
def train_image_only(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    loop = tqdm(loader, leave=True, desc="Train Image-Only")

    # Loader에서 tabular 데이터도 나오지만 무시(_)
    for images, _, labels in loop:
        images = images.to(device).float()
        labels = labels.to(device).float()

        optimizer.zero_grad()
        outputs = model(images) # 이미지만 입력
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        loop.set_description(f"Loss: {loss.item():.4f}")

    return running_loss / len(loader.dataset)

# 검증 함수
def validate_image_only(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_targets = []
    all_preds = []

    with torch.no_grad():
        for images, _, labels in loader:
            images = images.to(device).float()
            labels = labels.to(device).float()

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)

            all_targets.append(labels.cpu().numpy())
            all_preds.append(torch.sigmoid(outputs).cpu().numpy())

    all_targets = np.concatenate(all_targets)
    all_preds = np.concatenate(all_preds)

    try:
        auroc = roc_auc_score(all_targets, all_preds, average='macro')
    except:
        auroc = 0.5

    return running_loss / len(loader.dataset), auroc

# 메인 실행
best_auroc = 0.0
print("🚀 Image-Only Baseline Re-Training (Safe Aug + Dropout 0.5)...")

for epoch in range(NUM_EPOCHS):
    train_loss = train_image_only(model_img, train_loader, criterion, optimizer, device)
    val_loss, val_auroc = validate_image_only(model_img, val_loader, criterion, device)

    scheduler.step(val_auroc)

    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val AUROC: {val_auroc:.4f}")

    if val_auroc > best_auroc:
        best_auroc = val_auroc
        # 파일명 다르게 저장
        torch.save(model_img.state_dict(), os.path.join(BASE_DIR, "best_image_only_model_v2.pth"))
        print(f"✅ Best Baseline Saved! (AUROC: {val_auroc:.4f})")

    print("-" * 50)